# Lecture 17. PyTorch Regression

> Eunmi Kim    
 계산과학 프로그래밍 및 실습


---



In [16]:
#import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn import datasets

## 1. Data

**California housing dataset**


In [17]:
# load data
california_housing = datasets.fetch_california_housing(as_frame=True) #딕셔너리 형태

In [18]:
# key 값을 출력
print(california_housing.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])


In [19]:
print(california_housing.feature_names)
print(california_housing.target_names) #MedHouseVal(target)을 예측하는 것

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
['MedHouseVal']


In [20]:
california_housing.frame.head()
#frame : features&target

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [21]:
california_housing.data.head()
#input node=8,output node=1
#마지막 activation fcn = identity
#loss fcn=MSE
#이 세개는 데이터 주어졌을 때 결정된거~
#첫 레이어 노드 64, 두번째 레이어 노드 32 (총 3 layer)
#Linear1(8,64)+ReLU, Linear2(64,32)+ReLU, Linear3(32,1)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [10]:
california_housing.target.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64

In [9]:
print(california_housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [11]:
california_housing.frame.info() #info 쓰면 데이터 타입 볼 수 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [30]:
# to FloatTensor
data = torch.FloatTensor(california_housing.frame.values) #마지막 컬럼을 타겟으로

In [23]:
data.shape #총개수 20640, input:8 output:1

torch.Size([20640, 9])

In [33]:
torch.manual_seed(2023)
torch.randperm(5) #데이터 섞음 (random permutation)
#[0 1 2 3 4]->[0 2 3 1 4]

tensor([4, 2, 1, 0, 3])

In [37]:
# 데이터 셔플
num_data = data.shape[0]
index = torch.randperm(num_data)

data = data[index]

tensor([[   3.9741,   15.0000,    6.6457,  ...,   34.0600, -117.1400,
            1.4110],
        [   2.4688,   40.0000,    3.6472,  ...,   33.7900, -117.8500,
            2.3640],
        [   2.7273,   44.0000,    4.5625,  ...,   32.7300, -117.2300,
            2.6970],
        ...,
        [   3.7821,    5.0000,    5.5058,  ...,   36.7900, -119.8900,
            0.8040],
        [   6.2976,   52.0000,    7.6040,  ...,   34.1800, -118.2700,
            4.9840],
        [   4.3370,   34.0000,    5.9537,  ...,   33.9700, -117.4100,
            1.5780]])


In [38]:
# seperate input and target
x = data[:, :-1] #input
y = data[:, -1:] #output

print(x,y)
print(x.shape, y.shape)

tensor([[   3.9741,   15.0000,    6.6457,  ...,    2.5087,   34.0600,
         -117.1400],
        [   2.4688,   40.0000,    3.6472,  ...,    2.1254,   33.7900,
         -117.8500],
        [   2.7273,   44.0000,    4.5625,  ...,    1.9883,   32.7300,
         -117.2300],
        ...,
        [   3.7821,    5.0000,    5.5058,  ...,    3.1398,   36.7900,
         -119.8900],
        [   6.2976,   52.0000,    7.6040,  ...,    2.9023,   34.1800,
         -118.2700],
        [   4.3370,   34.0000,    5.9537,  ...,    2.4576,   33.9700,
         -117.4100]]) tensor([[1.4110],
        [2.3640],
        [2.6970],
        ...,
        [0.8040],
        [4.9840],
        [1.5780]])
torch.Size([20640, 8]) torch.Size([20640, 1])


In [28]:
# train, validation, test split
num_val = num_test = int(0.1*len(y))
num_train = num_data - num_test - num_val

train_x, val_x, test_x = x[:num_train], x[num_train: num_train+num_val], x[num_train+num_val :]
train_y, val_y, test_y = y[:num_train], y[num_train: num_train+num_val], y[num_train+num_val :]

In [39]:
# normalization
# validation과 test에 사용하는 데이터 또한 훈련 데이터에서 사용한 값을 이용하여 정규화해야 한다.
# training set에서 정규화
# 따라서 validation,test set도 training에서 사용한 애들로 shift,, normalization을 해줘야함
mean = train_x.mean(dim=0)
train_x -= mean
std = train_x.std(dim=0)
train_x /= std

val_x -= mean
val_x /= std

test_x -= mean
test_x /= std

## 2. Model

In [40]:
input_dim = train_x.shape[-1]

# nn layers
linear1 = nn.Linear(input_dim, 64, bias=True)
linear2 = nn.Linear(64, 32, bias=True)
linear3 = nn.Linear(32, 1, bias=True)
relu = nn.ReLU() #update할 weight이 없어서 걍 계속 같이 써도 ㄱㅊ

# sequential model
model = nn.Sequential(linear1, relu, linear2, relu, linear3)

# loss
loss_func = nn.MSELoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [41]:
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)


## 3. Training

In [42]:
epochs = 500

train_loss_list = []
val_loss_list = []

for epoch in range(epochs):
    optimizer.zero_grad()

    pred_y = model.forward(train_x)
    loss = loss_func(pred_y, train_y)
    train_loss_list.append(loss.item())

    # backward
    loss.backward()

    # weight update
    optimizer.step()

    # validation with no_grad (기울기 계산하지 않아, 계산량을 줄일 수 있다.)
    with torch.no_grad():
        val_pred = model.forward(val_x)
        val_loss = loss_func(val_pred, val_y)
        val_loss_list.append(val_loss)

    if (epoch+1) % 50 == 0:
        print("Epoch %5d: loss %.4f   val_loss %.4f" % (epoch+1, loss, val_loss))

Epoch    50: loss 0.3512   val_loss 0.3527
Epoch   100: loss 0.3406   val_loss 0.3077
Epoch   150: loss 0.2980   val_loss 0.3400
Epoch   200: loss 0.2734   val_loss 0.2989
Epoch   250: loss 0.2855   val_loss 0.2908
Epoch   300: loss 0.2559   val_loss 0.2929
Epoch   350: loss 0.2562   val_loss 0.2899
Epoch   400: loss 0.2697   val_loss 0.3096
Epoch   450: loss 0.2453   val_loss 0.2996
Epoch   500: loss 0.2472   val_loss 0.2751


In [ ]:
# plot loss
plt.plot(range(1, epochs+1), train_loss_list, range(1, epochs+1), val_loss_list)
plt.legend(['Train', 'Validation'])
plt.title('MSE Loss')
plt.show()

In [ ]:
# Test model
with torch.no_grad():
    pred_y = model.forward(test_x)
    test_loss = loss_func(pred_y, test_y)
    print(test_loss)

NameError: ignored

In [ ]:
torch.sqrt(test_loss)

NameError: ignored

In [ ]:
# 20개 예측 비교
with torch.no_grad():
    pred_y = model.forward(test_x[:20])
    print('prediction', pred_y.view(-1))
    print('target', test_y[:20].view(-1))

In [ ]:
plt.plot(test_y[:20].view(-1), '*', pred_y.view(-1))
plt.show()

## Weight Initialization

In [ ]:
input_dim = train_x.shape[-1]

# nn layers
linear1 = nn.Linear(input_dim, 64, bias=True)
linear2 = nn.Linear(64, 32, bias=True)
linear3 = nn.Linear(32, 1, bias=True)
relu = nn.ReLU()

# Kaiming He initialization
nn.init.kaiming_normal_(linear1.weight, nonlinearity='relu')
nn.init.kaiming_normal_(linear2.weight, nonlinearity='relu')
nn.init.kaiming_normal_(linear3.weight, nonlinearity='relu')

# sequential model
model = nn.Sequential(linear1, relu,
                      linear2, relu,
                      linear3)

# loss
loss_func = nn.MSELoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [ ]:
epochs = 500

train_loss_list_wi = []
val_loss_list_wi = []

for epoch in range(epochs):
    optimizer.zero_grad()

    pred_y = model.forward(train_x)
    loss = loss_func(pred_y, train_y)
    train_loss_list_wi.append(loss.item())

    # backward
    loss.backward()

    # weight update
    optimizer.step()

    # validation with no_grad
    with torch.no_grad():
        val_pred = model.forward(val_x)
        val_loss = loss_func(val_pred, val_y)
        val_loss_list_wi.append(val_loss)


    if (epoch+1) % 50 == 0:
        print("iteration %5d: loss %.4f   val_loss %.4f" % (epoch+1, loss, val_loss))

In [ ]:
# Test model
with torch.no_grad():
    test_loss = loss_func(model.forward(test_x), test_y)
    print(torch.sqrt(test_loss))

In [ ]:
# plot loss
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_loss_list, range(1, epochs+1), val_loss_list)
plt.legend(['Training', 'Validation'])
plt.title('Defaut Weight Initialization')
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), train_loss_list_wi, range(1, epochs+1), val_loss_list_wi)
plt.legend(['Training', 'Validation'])
plt.title('He Weight Initialization')
plt.show()